In [2]:
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [3]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x14965b070>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x14963f460>


In [4]:
# Gradient

loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3039, 0.2375, 0.3243],
        [0.3039, 0.2375, 0.3243],
        [0.3039, 0.2375, 0.3243],
        [0.3039, 0.2375, 0.3243],
        [0.3039, 0.2375, 0.3243]])
tensor([0.3039, 0.2375, 0.3243])


In [6]:
# 변화도 추적 멈추기

z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

True
False
False


변화도 추적을 멈춰야 하는 이유
- 고정된 매개변수(frozen parameter)는 사전 학습된 신경망을 미세조정
- 변화도를 추적하지 않는 텐서의 연산이 더 효율적 -> 연산 속도 향상

In [7]:
# 야코비안 곱(Jacobian Product)

inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


같은 인자로 backward를 두번 호출하면 변화도 값이 달라짐 -> 변화도를 누적해주기 때문